##### <font color='green'> TODO </font>:
- [x] make a generic function for Diagnostic category;
- [x] output chemical, protein, genetic biomarker types;
- how it can be linked to the organ (?);
- [x] do it with bulks;
- [x] find a tool to create a DB schema;
(https://online.visual-paradigm.com/pricing/, draw.io)
- create DB schema;
- add Complete Blood Count;
- calculate how it's expensive (gigas);
- find a good database (Aurora?);
- populate DB;
- cron job to do it automatically;
- what is the difference between these biomarkers (?);

## Findings:
- diagnostic-genetics - API does not work;
- diagnostic-chemoicals - API does not map correctly by marker_id;

## <font color='red'> BioMarkers </font>

category: {Diagnostic, Prognostic, Predictive, Exposure}

biomarker_type: {Protein, Karyotype}

In [361]:
%%capture

import os
import pandas as pd
import requests

pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

from pprint import pprint

In [258]:
! which python3

/Users/eolariu/miniconda3/envs/mybio/bin/python3


In [259]:
api_key_val = os.environ['API_KEY']

In [260]:
def output_all_conditions(url, category, biomarker_type, start_page, end_page):
    conditions = []
    
    for page in range(start_page, end_page):
        names = []
        parameters = {'api_key': api_key_val,
                      'category': category,
                      'biomarker_type': biomarker_type,
                      'page': page}

        r = requests.get(url, params=parameters)

        response = r.json()

        for key, value in response['biomarkers'].items():
            for biomarker in value:
                biomarker_list = [*biomarker.values()]
                conditions.append(biomarker_list)
    return conditions

## Diagnostic-Protein

In [230]:
start_page = 1
end_page = 72
# 72
category = 'Diagnostic'
biomarker_type = 'Protein'

url = 'http://markerdb.ca/api/v1/generalapi/generalrequest'

In [231]:
conditions_diagnostic_proteins = []
for i in range(start_page, end_page, batch):
    start_page_new = i
    end_page_new = i + batch
    if end_page < end_page_new:
        end_page_new = end_page + 1
    conditions_diagnostic_proteins_batch = output_all_conditions(url, 
                                                            category, 
                                                            biomarker_type, 
                                                            start_page_new, 
                                                            end_page_new)

    conditions_diagnostic_proteins += (conditions_diagnostic_proteins_batch)
    print(f'Pages from {start_page_new} - {end_page_new - 1} are successfully inserted')
print('Finished!')

Pages from 1 - 10 are successfully inserted
Pages from 11 - 20 are successfully inserted
Pages from 21 - 30 are successfully inserted
Pages from 31 - 40 are successfully inserted
Pages from 41 - 50 are successfully inserted
Pages from 51 - 60 are successfully inserted
Pages from 61 - 70 are successfully inserted
Pages from 71 - 72 are successfully inserted
Finished!


In [232]:
conditions_proteins = []
for i in conditions_diagnostic_proteins:
    # print(i[-1])
    conditions_proteins.append(i[-1])
conditions_proteins_unique = set(conditions_proteins)
print(len(conditions_proteins_unique))

91


In [235]:
conditions_proteins;

In [241]:
proteins_map = {}
for i in conditions_diagnostic_proteins:
    if i[0] not in proteins_map:
        proteins_map[i[0]] = i[1]

proteins_map;

In [365]:
conditions_proteins_dict = {}
for item in conditions_diagnostic_proteins:
    if item[-1] in conditions_proteins_dict:
        conditions_proteins_dict[item[-1]].append(item[0])
    else:
        conditions_proteins_dict[item[-1]] = [item[0]]

In [366]:
conditions_proteins_dict;

In [367]:
df_proteins = pd.DataFrame.from_dict(conditions_proteins_dict, 
                                     orient='index', 
                                     columns=[f'biomarker_protein_{i}' for i in range(1, 7)]
                                    )

In [368]:
df_proteins

,biomarker_protein_1,biomarker_protein_2,biomarker_protein_3,biomarker_protein_4,biomarker_protein_5,biomarker_protein_6
Sarcoidosis,Angiotensin-converting enzyme,None,None,None,None,None
Diabetes Mellitus Type 2,Glycated hemoglobin,C-reactive protein,None,None,None,None
Streptococcus Infection,Anti-DNase B Antibodies,Anti-Streptolysin antibodies,None,None,None,None
Chronic Kidney Disease,Transthyretin,Albumin,Erythropoietin,None,None,None
Malnutrition,Transthyretin,Albumin,None,None,None,None
Acute Kidney Injury,Cystatin-C,N-acetyl-beta-D-glucosaminidase,Vascular Endothelial Growth Factor,Neutrophil Gelatinase associated lipocalin,Hepatitis A virus cellular receptor 1,None
Acute Pancreatitis,Amylase,None,None,None,None,None
Tuberculosis,Adenosine deaminase,None,None,None,None,None
Osteoporosis,N-terminal propeptide of type 1 procollagen,N-terminal telopeptide,C-terminal telopeptide,None,None,None
Osteomalacia,N-terminal telopeptide,None,None,None,None,None


In [369]:
df_proteins.reset_index(level=0, inplace=True)
df_proteins = df_proteins.rename({'index': 'condition_name'}, axis=1)

In [370]:
df_proteins[:20]

,condition_name,biomarker_protein_1,biomarker_protein_2,biomarker_protein_3,biomarker_protein_4,biomarker_protein_5,biomarker_protein_6
0,Sarcoidosis,Angiotensin-converting enzyme,None,None,None,None,None
1,Diabetes Mellitus Type 2,Glycated hemoglobin,C-reactive protein,None,None,None,None
2,Streptococcus Infection,Anti-DNase B Antibodies,Anti-Streptolysin antibodies,None,None,None,None
3,Chronic Kidney Disease,Transthyretin,Albumin,Erythropoietin,None,None,None
4,Malnutrition,Transthyretin,Albumin,None,None,None,None
5,Acute Kidney Injury,Cystatin-C,N-acetyl-beta-D-glucosaminidase,Vascular Endothelial Growth Factor,Neutrophil Gelatinase associated lipocalin,Hepatitis A virus cellular receptor 1,None
6,Acute Pancreatitis,Amylase,None,None,None,None,None
7,Tuberculosis,Adenosine deaminase,None,None,None,None,None
8,Osteoporosis,N-terminal propeptide of type 1 procollagen,N-terminal telopeptide,C-terminal telopeptide,None,None,None
9,Osteomalacia,N-terminal telopeptide,None,None,None,None,None


In [371]:
# df_proteins[df_proteins['condition_name'] == 'Polycystic Ovary Syndrome']
df_proteins[df_proteins['condition_name'] == 'Diabetes Mellitus Type 2']

,condition_name,biomarker_protein_1,biomarker_protein_2,biomarker_protein_3,biomarker_protein_4,biomarker_protein_5,biomarker_protein_6
1,Diabetes Mellitus Type 2,Glycated hemoglobin,C-reactive protein,None,None,None,None


In [372]:
df_proteins[11:20];

In [299]:
conditions_proteins_list = list(conditions_proteins_dict.keys())
conditions_proteins_list;

In [385]:
df_proteins.to_csv('conditions.csv', index=False)

## Protein biomarkers search by ID.

In [322]:
proteins_map;

# how to slice a dictionary.
# import itertools
# proteins_map_sliced = dict(itertools.islice(proteins_map.items(), 2))
# proteins_map_sliced

In [319]:
protein_biomarkers_dict = {}

# base = 'MDB'
# start_marker_number = 873
# end_marker_number = 56533

# for number in range(start_marker_number-1, end_marker_number):
# number_str = base + str(number).zfill(8)
for biomarker_name, biomarker_id in proteins_map.items():
    parameters = {'api_key': api_key_val,
                  'markerdb_id': biomarker_id}

    r = requests.get('http://markerdb.ca/api/v1/proteinapi/proteinrequest', params=parameters)
    try:
        response = r.json()['protein']
        general_function = response['general_function']
        protein_biomarkers_dict[biomarker_name] = [biomarker_id, general_function]
        # print(f'{biomarker_name} is added 🤗')
    except KeyError:
        pass

In [373]:
protein_biomarkers_dict;

In [376]:
df_proteins_biomarkers = pd.DataFrame.from_dict(protein_biomarkers_dict, 
                                     orient='index', 
                                     columns=['biomarker_id', 'description']
                                    )
df_proteins_biomarkers[:3];

In [377]:
df_proteins_biomarkers.reset_index(level=0, inplace=True)
df_proteins_biomarkers = df_proteins_biomarkers.rename({'index': 'biomarker_name'}, axis=1)

In [378]:
biomarker_name = 'Angiotensin-converting enzyme'
description = df_proteins_biomarkers[df_proteins_biomarkers['biomarker_name'] == biomarker_name]['description']
description

0    Converts angiotensin I to angiotensin II by release of the terminal His-Leu, this results in an increase of the vasoconstrictor activity of angiotensin. Also able to inactivate bradykinin, a potent vasodilator. Has also a glycosidase activity which releases GPI-anchored proteins from the membrane by cleaving the mannose linkage in the GPI moiety. [Uniprot]
Name: description, dtype: object

In [383]:
df_proteins_biomarkers[:2]

,biomarker_name,biomarker_id,description
0,Angiotensin-converting enzyme,MDB00000874,"Converts angiotensin I to angiotensin II by release of the terminal His-Leu, this results in an increase of the vasoconstrictor activity of angiotensin. Also able to inactivate bradykinin, a potent vasodilator. Has also a glycosidase activity which releases GPI-anchored proteins from the membrane by cleaving the mannose linkage in the GPI moiety. [Uniprot]"
1,Glycated hemoglobin,MDB00000873,Involved in oxygen transport from the lung to the various peripheral tissues. [Uniprot]


In [384]:
df_proteins_biomarkers.to_csv('biomarkers.csv', index=False)

## Diagnostic-Karyotype

In [198]:
start_page = 1
end_page = 10
# 
category = 'Diagnostic'
biomarker_type = 'Karyotype'

url = 'http://markerdb.ca/api/v1/generalapi/generalrequest'

In [199]:
conditions_diagnostic_karyotype = []
for i in range(start_page, end_page, batch):
    start_page_new = i
    end_page_new = i + batch
    if end_page < end_page_new:
        end_page_new = end_page + 1
    conditions_diagnostic_karyotype_batch = output_all_conditions(url, 
                                                            category, 
                                                            biomarker_type, 
                                                            start_page_new, 
                                                            end_page_new)

    conditions_diagnostic_karyotype += (conditions_diagnostic_karyotype_batch)
    print(f'Pages from {start_page_new} - {end_page_new - 1} are successfully inserted')
print('Finished!')

Pages from 1 - 10 are successfully inserted
Finished!


In [294]:
conditions_diagnostic_karyotype;

In [111]:
# organs should be functionioning (bars how good they do);

## <font color='red'> Complete Blood Count </font>

## <font color='red'> Female Reproductive conditions and their biomarkers. </font>

- Autoimmune diseases(?)
- Alopecia areata
- Antiphospholipid (an-teye-FOSS-foh-lip-ihd) antibody syndrome (aPL)
- Autoimmune hepatitis
- Bacterial vaginosis
- Bladder pain syndrome (interstitial cystitis)
- Breast cancer
- Carpal tunnel syndrome
- Celiac disease
- Cervical cancer
- Chlamydia
- Chronic fatigue syndrome/myalgic encephalomyelitis (ME/CFS)
- Chronic obstructive pulmonary disease (COPD)
- Diabetes
- Emergency birth control/emergency contraception
- Endometriosis
- Female genital mutilation or cutting (FGM/C)
- Fibroids (uterine)
- Fibromyalgia
- Genital herpes
- Genital warts
- Gonorrhea
- Graves' disease
- Guillain-Barre
- Hashimoto's disease
- Heart disease
- Hepatitis
- Herpes
- Hemolytic anemia
- HIV and AIDS
- Hysterectomy
- Infertility
- Inflammatory bowel disease (IBD)
- Insomnia
- Interstitial cystitis (bladder pain syndrome)
- Iron-deficiency anemia
- Irritable bowel syndrome (IBS)
- Lupus
- Menopause
- Myalgic encephalomyelitis/chronic fatigue syndrome (ME/CFS)
- Myasthenia gravis
- Neonatal Abstinence Syndrome/Opioid Withdrawal in Infants
- Osteoporosis
- Ovarian cancer
- Ovarian cysts
- Ovarian syndrome (PCOS or polycystic ovary syndrome)
- Pap and HPV tests
- Polycystic ovary syndrome (PCOS)
- Pelvic inflammatory disease (PID)
- Pelvic organ prolapse
- Pregnancy
- Postpartum depression
- Premenstrual syndrome (PMS)
- Sexually transmitted infections (STDs, STIs)
- Sickle cell disease
- Stroke
- Syphilis
- Thyroid disease
- Trichomoniasis
- Urinary incontinence
- Urinary tract infection (UTI)
- Uterine cancer
- Uterine fibroids
- Vaginal yeast infections
- Varicose veins and spider veins
- Viral hepatitis
- Weight loss (and overweight and obesity)
- Yeast infections

autoimmune diseases

- Alopecia areata;
- Antiphospholipid (an-teye-FOSS-foh-lip-ihd) antibody syndrome (aPL);
- Autoimmune hepatitis;
- Celiac disease;
- Diabetes type 1;
- Graves' disease (overactive thyroid);
- Guillain-Barre (GEE-yahn bah-RAY) syndrome;
- Hashimoto's (hah-shee-MOH-tohz) disease (underactive thyroid);
- Hemolytic anemia;
- Idiopathic thrombocytopenic purpura;
- Inflammatory bowel disease (IBD);
- Inflammatory myopathies;
- Multiple sclerosis;
- Myasthenia gravis;
- Primary biliary cirrhosis;
- Psoriasis;
- Rheumatoid arthritis;
- Scleroderma;
- Sjögren's (SHOH-grins) syndrome;
- Systemic lupus erythematosus;
- Vitiligo;

- urinary system disorder;
- respiratory system disorder;
- Immune System Disorder;
- Hematological or Lymphatic System Disorder;
- Female Reproductive System Disorder
- Germ-Line Disorder;
- Bacterial Infection
- Cancer
- Cardiovascular System Disorder
- Digestive System Disorder
- Endocrine System Disorder

In [220]:
disorders_types = [
    [1, "urinary system disorder"],
    [2, "respiratory system disorder"],
    [3, "immune System Disorder"],
    [4, "Hematological or Lymphatic System Disorder"],
    [5, "Female Reproductive System Disorder"],
    [7, "Bacterial Infection"],
    [8, "Cancer"],
    [9, "Cardiovascular System Disorder"],
    [10, "Digestive System Disorder"],
    [11, "Endocrine System Disorder"]]

In [221]:
disorder_type_df = pd.DataFrame(disorders_types, columns=['disorder_type_id', 'disorder_name'])

In [222]:
disorder_type_df

,disorder_type_id,disorder_name
0,1,urinary system disorder
1,2,respiratory system disorder
2,3,immune System Disorder
3,4,Hematological or Lymphatic System Disorder
4,5,Female Reproductive System Disorder
5,7,Bacterial Infection
6,8,Cancer
7,9,Cardiovascular System Disorder
8,10,Digestive System Disorder
9,11,Endocrine System Disorder


In [226]:
organs = [
    [1, "liver"],
    [2, "bowel"],
    [3, "brain"],
    [4, "uterus"],
    [5, "ovaries"],
    [6, "thyroid"],
    [7, "bladder"]]

In [227]:
organs_df = pd.DataFrame(organs, columns=['organ_id', 'organ'])

In [256]:
organs_df

,organ_id,organ
0,1,liver
1,2,bowel
2,3,brain
3,4,uterus
4,5,ovaries
5,6,thyroid
6,7,bladder
